### In this notebook, I will use regularized expressions to filter out noise from documents in google cloud storage. I will calculate the similarity score between files sent by customers. The similarity score will flag files that may be duplicates 

In [1]:
%%capture


# Consolidated pip install commands
! pip install python-docx gcsfs pandas nltk PyMuPDF Levenshtein openpyxl

# Import statements
import gcsfs
import fitz  # PyMuPDF
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from docx import Document
from io import BytesIO
import warnings
import numpy as np
from collections import Counter, defaultdict
from nltk.corpus import stopwords
from string import punctuation
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from google.cloud import bigquery
from google.cloud import storage
from tqdm import tqdm
import Levenshtein as lev
from itertools import combinations


# I will convert the confirmed duplicates into ngrams and then run my ngram similarity matrix on those documents. 

In [2]:
bq_query = """

WITH string_list AS (
SELECT 'cc/000-02-548066/SIG1835_N.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-548073/Tamron 70-180mm .pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-548079/PANA1025.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-548086/Sony90.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-549000/FUji1655.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-549310/OLY714.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-549390/PANA1025.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-549449/Sony90.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-549299/CAN24120.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-549305/Leica18.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-549316/SIG1835_N.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-549371/Tamron 70-180mm .pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-420369/CAN24240.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-420635/FUji1655.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-420644/Leica18.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-420650/OLY75.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-420653/SIG1835.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-420659/Tamron_1728.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-420668/PANA1025.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-420677/Sony20.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-444352/CAN24240.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-444361/FUji1655.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-444371/Leica18.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-444386/OLY75.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-444396/SIG105.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-444409/Tamron 70-180mm .pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-444418/PANA1025.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-444425/Sony20.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-448358/CAN24240.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-448369/FUji1655.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-448399/Leica18.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-448471/OLY75.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-157475/NIK2470Z.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-163427/PEN1530.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-163435/TAM150500.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-163440/SIG105.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-163449/OLY75.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-163460/Leica18.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-163469/FUji1655.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-163487/CAN_R.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-168080/NIK241K.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-168099/PEN1530.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-168114/TAM20200.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-168124/SIG2470.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-168138/OLY12100.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-168150/Leica1856.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-168159/FUJIXE4.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-168170/CAN35.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-178686/NIK241K.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-178903/PEN1530.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-178912/TAM20200.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-178920/SIG2470.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-178944/OLY12100.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-178959/Leica1856.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-178974/FUJIXE4.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-178984/CAN35.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-181193/CAN35.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-181203/FUJIXE4.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-181209/Leica1856.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-181212/OLY12100.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-181222/SIG1835.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-181230/TAM150500.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-424664/PEN2470PDF.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-554516/OLY714.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-554525/jvc.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-554555/PEN1530.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-570137/SIG2470_1K.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-570142/Nik Z5.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-387948/Pana818.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-387964/SAMP95XDBKB1.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-391973/sny24mm.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-398164/jvc.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-398207/NIKZ5.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-424720/TAM2875PDF.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-437466/PANA1025ECT.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-437480/OLY12100ECT.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-558613/CAN_R.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-612300/pentax.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-612326/jvc.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-614810/OLY714.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-614814/BMJ.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-636244/PEN2470.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-636247/jvc.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-636250/BMJ.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-636253/OLY12100ECT.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-679403/FUJIXE4.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-679410/NIK2470Z.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-679414/Pana818.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-679437/SIG1835_N.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-437426/TAM2875_PCD.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-570008/SIG105_PCD.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-430412/BMJ_PCD.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-430418/CANEOSR_PCD.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-430430/NIKZ5_PCD.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-433069/Samsung_PCD.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-434865/SIG105_PCD.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-551375/CAN_R_procam.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-551380/SNYa7rII_PCD.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-551395/jvc.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-551508/PANA1025.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-596951/SIG2470_1K.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-596964/Samsung.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-596970/Sony90.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-596975/CAN24120.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-644931/TAM2875.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-644941/CAN_R.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-644944/Samsung.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-644958/SIG105art_1K.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-679454/TAM2875_PCD.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-679459/NIK2470Zn.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-679463/FUji1655.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-679469/Sony90.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-825813/PEN1118.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-858280/Leica18.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-858284/SIG2470_.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-858293/FUji1655.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-858296/Pana818.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-871372/NIKON z6.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-871375/SAMP95XDBKB1.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-871378/OLY12100.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-871381/PEN1118.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-894953/BMJ_T.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-894956/jvc.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-894959/Surface Pro 7.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-825803/OLY714.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-727598/OLY714RT.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-727601/PEN1118RT.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-727604/NIK2470ZRT.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-727609/CAN_RRT.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-827586/NIKZ5.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-827596/CAN_R6.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-894965/TAM70180.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-006494/CAN_R6.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-007107/FUji1655T.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-007114/Leica23T.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-007117/PEN1118T.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-007120/NIKKOR Z 50T.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-007123/OLY75T.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-007156/SIG1835T.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-007159/TAM70180T.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-015663/CAN35MM.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-015666/FUJI16.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-015672/PEN1530.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-015677/NIKKOR Z 50T.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-015680/OLY75T.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-015683/SIG1835T.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-015686/TAM70180T.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-018213/CAN35MM.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-018217/FUJI16.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-018220/Leica23T.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-880708/OLY75.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-716442/Pana818.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-725330/CAN24120.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-725333/BMJ.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-725336/sny24mm.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-837194/CAN24120.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-837220/Pana818.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-837234/BMJ.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-837252/sny24mm.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-858117/FUJIXE4.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-858123/SIG105.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-858128/PEN1530.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-858135/NIK241K.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-871384/CAN24240.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-871387/Pana818.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-871390/BMJ.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-871396/sny24mm.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-880714/Tamron 70-180mm .pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-894980/Surface Pro 7.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-894986/jvc.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-913471/sny24mm.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-913488/BMJ_.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-913517/Leica18.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-913535/SIG105.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-917675/JVC_L.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-917678/Tamron 70-180mm .pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-864845/XF35.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-865867/OLY75.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-865879/D5600.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-866575/D5600.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-905526/BMJ.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-905532/CAN24240.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-905538/FUji1655.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-905541/jvc.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-913386/Leica18_A.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-913421/NIK2470Z_A.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-913424/OLY75_A.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-913434/SIG1835_A.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-997693/CAN24240.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-997698/FUji1655.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-997702/Leica18.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-997705/OLY75_A.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-997710/Pana818.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-01-997714/Tamron 70-180mm .pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-004624/SIG1835_A.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-004628/NIK241K.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-007162/CAN24240.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-007165/FUJIXE4.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-007168/Leica23.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-007173/OLY75.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-007176/PANA1025.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-007180/Tamron 70-180mm .pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-012487/SIG2470P.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-012496/NIK2470ZP.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-046757/Canon T8i.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-046760/FUJI XF35.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-406227/SIG1835.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-288445/Canon T8i.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-363734/Canon T8i.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-363743/FUJI XF35.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-363746/Leica23.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-363752/OLY75.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-365786/SIG2470.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-365791/TAM20.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-365802/Pana818.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-365805/sny24mm.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-371742/CAN_R.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-371745/FUJIXE4.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-371748/Leica23.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-371751/OLY75.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-373524/SIG2470.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-373527/TAM20.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-373532/Pana818.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-373539/sny24mm.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-377071/CAN_R.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-377074/FUJIXE4.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-377077/Leica23.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-377080/OLY75.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-378211/SIG2470.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-378218/TAM20.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-378228/Pana818.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-378241/sny24mm.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-386222/CAN24240.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-386225/FUji16.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-386228/OLY75.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-386231/SIG1835.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-386234/TAM20.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-386237/Pana818.pdf'  AS string_to_match UNION ALL
SELECT 'cc/000-02-386240/Sony20.pdf'  AS string_to_match

)

SELECT original_table.BlobName, original_table.ModifiedContentType as ContentType

FROM gcp-bia-mle-dev.test.GWArchiveBlobAndContent AS original_table
JOIN string_list
ON original_table.BlobName LIKE CONCAT(string_list.string_to_match, '%')

"""

In [3]:

df = pd.read_gbq(bq_query)



# Initialize GCS File System

In [4]:
gcs_file_system = gcsfs.GCSFileSystem(project="gcp-bia-mle-dev")

gcs_directory_path = "gwclaimmigration"



## This code is designed to process PDF files stored in a Google Cloud Storage bucket, extract their text, and then generate n-grams from this text, storing the results in a pandas DataFrame.

 The code chunk below applies the process_pdf function to each PDF file whose name in the 'BlobName' column starts with 'cc/'. extracts 5-grams and stores them in a new column 'Ngrams'. 
If the file name does not start with 'cc/', it assigns None to that row in the 'Ngrams' column.



In [5]:

# Initialize GCS File System
gcs_file_system = gcsfs.GCSFileSystem(project="gcp-bia-mle-dev")

# Specify your GCS bucket name
bucket_name = "gwclaimmigration"  

# Function to generate n-grams
def generate_ngrams(text, n): # takes a string text and an integer n,
    words = word_tokenize(text) #tokenizes the text into words,
    return [' '.join(gram) for gram in ngrams(words, n)] #generates n-grams of size n. An n-gram is a contiguous sequence of n words from the text.

# Function to process a single PDF file
def process_pdf(blobname):
    #print(blobname)
    full_path = f"{bucket_name}/{blobname}" #Constructs the full path to the file in the GCS bucket.
    with gcs_file_system.open(full_path, "rb") as f: #Opens the PDF file from GCS as a byte stream.
        pdf_file_bytes = BytesIO(f.read()) #Uses fitz (PyMuPDF) to read the PDF file from the byte stream.
    document = fitz.open("pdf", pdf_file_bytes) #Extracts the text from each page of the PDF and combines it into a single string.
    full_text = ' '.join([page.get_text() for page in document])
    return generate_ngrams(full_text, n=5)  


df['Ngrams'] = df['BlobName'].apply(lambda x: process_pdf(x) if x.startswith('cc/') else None) 



In [6]:
bucket_name = 'gwclaimmigration'
client = storage.Client()
bucket = client.get_bucket(bucket_name)

The code chunk below creates three lists. ACCEPTED_TYPES = ['application/pdf'] creates a list named ACCEPTED_TYPES containing a single string: 'application/pdf'. This suggests that the script is intended to process files of type PDF, as 'application/pdf' is the MIME type for PDF files. This list may be used to filter or validate the types of files being processed.  byte_rep_list. This list is not used in the provided snippet, but it suggests that there might be some operations related to byte representations of data in other parts of the script.  hash_rep_list. This list is used to store hash representations (specifically MD5 hashes) of the data being processed.

In [7]:
#allow the script to process only blobs whose content type is listed in ACCEPTED_TYPES.
ACCEPTED_TYPES = ['application/pdf'] 
byte_rep_list = []
hash_rep_list = []
#iterrows() function populates current row's index and data,

for index, row in df.iterrows():
#extracts the value from the column 'BlobName' of the current row and assigns it to the variable blob_name.
    blob_name = row['BlobName']
#extracts the 'ContentType' of the blob from df.Used to check the type of file (e.g., PDF, image, etc.).
    content_type = row['ContentType']
    # if content_type in ACCEPTED_TYPES:
    blob = bucket.get_blob(blob_name)
    #retrieves the blob from GCS using its name (blob_name). 
    #The variable bucket is assumed to be a reference to a specific storage bucket in GCS. 
    #The retrieved blob object is stored in the variable blob.
    hash_rep = blob.md5_hash
    #This hash is likely used as a unique identifier or checksum for the data in the blob.
    hash_rep_list.append(hash_rep)
    #The obtained MD5 hash (hash_rep) is appended to the list hash_rep_list.
    #This builds a list of hashes for all processed blobs.

df['HashRep'] = hash_rep_list
# a new column 'HashRep' is added to the DataFrame df, containing the list of MD5 hashes. 
#This associates each row in the DataFrame with the corresponding hash of the blob it represents.


In [8]:
duplicates_in_hash = df[df.duplicated(subset='HashRep', keep=False)]  
grouped = duplicates_in_hash.groupby('HashRep')['BlobName'].apply(list)

# Get the pairs of names that correspond to duplicate values in column HashRep
duplicate_name_pairs = [tuple(group) for group in grouped if len(group) > 1]

In [9]:
duplicate_name = pd.DataFrame(duplicate_name_pairs)

In [10]:
# The above files are exact duplicates. Therefore their similarity score should be 0. 

In [11]:
#  Initializes an empty dictionary to store frequencies of n-grams.


def ngram_frequency(ngrams):
    freq_dict = dict()
    for ngram in ngrams: #Iterates over each n-gram in the provided list. 
        if ngram in freq_dict.keys(): #Checks if the current n-gram is already in the dictionary.
          freq_dict[ngram] += 1 #If the n-gram is already in the dictionary, increments its count by 1.
        else:
          freq_dict[ngram] = 1 # If the n-gram is not in the dictionary, adds it with a count of 1.
    return freq_dict
    

In [12]:
def ngram_similarity(freq1, freq2):
  if len(freq1) == 0 or len (freq2)== 0: #Checks if either of the frequency dictionaries is empty.
    return 1 # If either is empty, returns 1 (indicating no similarity).
  freq1_total = 0 #Initialize several variables to store totals and differences:
  freq2_total = 0
  abs_diff = 0
  freq1_keyset = set(freq1.keys()) # Create sets of keys (n-grams) from both dictionaries.
  freq2_keyset = set(freq2.keys())
  for freq1_ngram in freq1_keyset: # Iterate over each n-gram in the first frequency set.
    freq1_total += freq1[freq1_ngram] #Add the frequency of this n-gram to the total for the first set.
    if freq1_ngram in freq2_keyset: # Check if this n-gram is also in the second set.
      freq2_total += freq2[freq1_ngram] # If yes, add its frequency to the total for the second set.
      abs_diff += abs(freq1[freq1_ngram] - freq2[freq1_ngram])
        #Calculate the absolute difference in frequencies for this n-gram and add to the total difference.
    else:
      abs_diff += freq1[freq1_ngram] #If the n-gram is not in the second set, add its frequency to the total difference.
  ngrams_in_2not1 = freq2_keyset.difference(freq1_keyset) #Find n-grams that are in the second set but not in the first.
  for ngram in ngrams_in_2not1: #Iterate over these n-grams.
    freq2_total += freq2[ngram] #Add their frequencies to the total for the second set.
    abs_diff += freq2[ngram] #Add their frequencies to the total difference.
  return abs_diff / (freq1_total + freq2_total) 
#Calculate and return the similarity measure. It is the total absolute difference divided by the sum of total frequencies in both sets.

## Inside the nested loops, the ngram_similarity function is called with two arguments: frequencies[a_row] and frequencies[b_row]. These are elements from the frequencies list created earlier, corresponding to the frequencies of the n-grams in the a_rowth and b_rowth positions.ngram_similarity() presumably calculates the similarity between two n-grams based on their frequencies. The result of this calculation is stored in the similarities matrix at position [a_row, b_row]. This means the similarity score for each pair of n-grams is stored in the corresponding cell of the matrix.

In [13]:
Ngramdf = df

n_rows = Ngramdf.shape[0]

frequencies = [ngram_frequency(Ngramdf['Ngrams'].iloc[i]) for i in range(n_rows)]

similarities = np.zeros((n_rows, n_rows))
for a_row in range(n_rows):
  for b_row in range(n_rows):
    similarities[a_row, b_row] = ngram_similarity(frequencies[a_row], frequencies[b_row])

## This line sets a variable suspicion_threshold to a value of 0.01. It appears to define a threshold for determining what is considered 'suspicious' in the context of this code. The specific meaning of 'suspicious' would depend on what the similarities matrix represents. Here, the code is creating a Boolean matrix called suspicious_similarities. similarities < suspicion_threshold is a conditional statement that checks each element in the similarities matrix to see if it is less than the suspicion_threshold value. For each element in similarities, if the element's value is less than 0.01, the corresponding element in suspicious_similarities will be True, otherwise False.This line essentially identifies all elements in the similarities matrix that are below the specified threshold.

In [14]:
# Similarities is a matrix 
suspicion_threshold = 0.125
suspicious_similarities = similarities < suspicion_threshold

#np.where(suspicious_similarities) is a function from the NumPy library. 
#It returns the indices of the True elements in suspicious_similarities. These indices are the locations in the similarities matrix where the values are less than the suspicion_threshold.
#The * operator is used to unpack the tuple of arrays returned by np.where.
#zip(*array) is then used to pair these indices, converting the row indices and column indices into pairs of coordinates.
#Finally, list() converts these pairs of coordinates into a list, which is stored in suspicious_idxs.

suspicious_idxs = list(zip(*np.where(suspicious_similarities)))

This line of code is a list comprehension that filters and restructures the contents of the suspicious_idxs list. 
suspicious_idx = [(i,j) for (i,j) in suspicious_idxs if i != j]:
suspicious_idx is a new list being created.
[(i,j) for (i,j) in suspicious_idxs if i != j] is the list comprehension. It iterates over each tuple (i, j) in the suspicious_idxs list.
suspicious_idxs is a list of tuples, where each tuple contains two indices (i, j). These indices likely represent positions in a matrix (the similarities matrix from your previous code snippets) that met a certain criterion (values below the suspicion_threshold).
if i != j is a conditional statement within the list comprehension. It checks whether the two elements of each tuple (i, j) are not equal.
Only the tuples where i is not equal to j are included in the new list suspicious_idx. This means that the code is filtering out all the pairs of indices where the row and column are the same (diagonal elements of the matrix), focusing only on the elements where the row and column indices differ.
this line creates a new list suspicious_idx that contains only the coordinates of elements in the similarities matrix that were previously identified as suspicious (i.e., their value is below the suspicion_threshold). This implies that the code is specifically interested in pairs of different items (as indicated by the row and column indices) that have a suspicious similarity score.

In [15]:
suspicious_idx = [(i,j) for (i,j) in suspicious_idxs if i != j]


In [16]:
len(suspicious_idxs) 

589

## For a single unique file, if there are three identical files there would be 6 pairs. 

In [17]:
suspicious_files = {}
for idxs in suspicious_idx:
  n0 = df['BlobName'].iloc[idxs[0]]
  n1 = df['BlobName'].iloc[idxs[1]]
  matching_keys = []
  for k in suspicious_files.keys():
    if (n0 in suspicious_files[k] or n1 in suspicious_files[k]):
      suspicious_files[k].add(n0)
      suspicious_files[k].add(n1)
      matching_keys.append(k)
  if len(matching_keys) > 0:
    for k in matching_keys[1:]:
      suspicious_files[matching_keys[0]].update(suspicious_files[k])
      del suspicious_files[k]
  else:
    suspicious_files[n0] = {n0, n1}

suspicious_df = pd.DataFrame([list(suspicious_files[key]) for key in suspicious_files.keys()])

## This dictionary has a list of the duplicate files based on the ngram similarity

In [18]:
suspicious_df.shape

(63, 10)

In [19]:
suspicious_df.to_csv('onetwofive_df.csv',index=True)

In [20]:
suspicious_df.head(46)

,0,1,2,3,4,5,6,7,8,9
0,cc/000-02-004628/NIK241K.pdf_2375003.pdf,cc/000-01-858135/NIK241K.pdf_2173342.pdf,None,None,None,None,None,None,None,None
1,cc/000-02-363746/Leica23.pdf_2845584.pdf,cc/000-02-377077/Leica23.pdf_2865741.pdf,cc/000-02-007168/Leica23.pdf_2378642.pdf,cc/000-02-371748/Leica23.pdf_2857971.pdf,None,None,None,None,None,None
2,cc/000-01-437480/OLY12100ECT.pdf_1664877.pdf,cc/000-01-636253/OLY12100ECT.pdf_1902666.pdf,None,None,None,None,None,None,None,None
3,cc/000-01-997705/OLY75_A.pdf_2368110.pdf,cc/000-02-420650/OLY75.pdf_2933963.pdf,cc/000-02-363752/OLY75.pdf_2845586.pdf,cc/000-02-444386/OLY75.pdf_2961552.pdf,cc/000-02-386228/OLY75.pdf_2878379.pdf,cc/000-02-377080/OLY75.pdf_2865742.pdf,cc/000-02-448471/OLY75.pdf_2969283.pdf,cc/000-02-007173/OLY75.pdf_2378643.pdf,cc/000-02-371751/OLY75.pdf_2857972.pdf,cc/000-01-913424/OLY75_A.pdf_2245677.pdf
4,cc/000-01-837220/Pana818.pdf_2140403.pdf,cc/000-01-871387/Pana818.pdf_2193002.pdf,None,None,None,None,None,None,None,None
5,cc/000-02-371742/CAN_R.pdf_2857969.pdf,cc/000-02-377071/CAN_R.pdf_2865739.pdf,None,None,None,None,None,None,None,None
6,cc/000-01-837252/sny24mm.pdf_2140443.pdf,cc/000-01-725336/sny24mm.pdf_2012074.pdf,cc/000-01-913471/sny24mm.pdf_2245822.pdf,cc/000-01-871396/sny24mm.pdf_2193004.pdf,None,None,None,None,None,None
7,cc/000-02-386231/SIG1835.pdf_2878380.pdf,cc/000-02-406227/SIG1835.pdf_2906782.pdf,None,None,None,None,None,None,None,None
8,cc/000-02-168138/OLY12100.pdf_2578155.pdf,cc/000-02-181212/OLY12100.pdf_2595207.pdf,cc/000-02-178944/OLY12100.pdf_2590847.pdf,None,None,None,None,None,None,None
9,cc/000-02-007156/SIG1835T.pdf_2378636.pdf,cc/000-02-015683/SIG1835T.pdf_2387495.pdf,None,None,None,None,None,None,None,None


In [21]:
duplicates_in_hash = df[df.duplicated(subset='HashRep', keep=False)]  
grouped = duplicates_in_hash.groupby('HashRep')['BlobName'].apply(list)

# Get the pairs of names that correspond to duplicate values in column HashRep
duplicate_name_pairs = [tuple(group) for group in grouped if len(group) > 1]

print("Pairs of names corresponding to duplicates in column 'HashRep':")
# print(duplicate_name_pairs)
print(len(duplicate_name_pairs))
item_counter = 0
for tup in duplicate_name_pairs:
    for item in tup:
        item_counter+=1

print(f'number of matches found: {item_counter}')


Pairs of names corresponding to duplicates in column 'HashRep':
44
number of matches found: 102


In [22]:
dupe = pd.read_csv("dupe.csv")


In [23]:
duplicates_in_hash.head()

,BlobName,ContentType,Ngrams,HashRep
3,cc/000-01-636253/OLY12100ECT.pdf_1902666.pdf,application/pdf,"[Laikok Photographic Products ( HK, Photograph...",mpXnNRP054Za3Z0O6Fm7Yw==
4,cc/000-01-913424/OLY75_A.pdf_2245677.pdf,application/pdf,"[Union Camera Hong Kong Limited, Camera Hong K...",Ogvn13NkrYhWVYWh6g5yDA==
6,cc/000-01-871387/Pana818.pdf_2193002.pdf,application/pdf,"[Laikok Photographic Products ( HK, Photograph...",GgRtQxFw6NOiHTpw9+Ks+A==
8,cc/000-02-377071/CAN_R.pdf_2865739.pdf,application/pdf,"[Union Camera Hong Kong Limited, Camera Hong K...",DSJDJFRwbN+R1Vdkre0o1A==
11,cc/000-02-386231/SIG1835.pdf_2878380.pdf,application/pdf,"[Union Camera Hong Kong Limited, Camera Hong K...",XKjPa1P7TsdcpbztjfXucg==


In [24]:
duplicates_in_hash.shape

(102, 4)

In [25]:
duplicates_in_hash.to_csv('duplicates_in_hash.csv',index=True)

In [26]:
result = pd.concat([dupe, suspicious_df], axis=1)


In [27]:
result.head()

,('cc/000-02-549371/Tamron 70-180mm .pdf_3123750.pdf','cc/000-02-548073/Tamron 70-180mm .pdf_3121278.pdf'),Unnamed: 2,Unnamed: 3,0,1,2,3,4,5,6,7,8,9
0,('cc/000-02-018217/FUJI16.pdf_2393514.pdf','cc/000-02-015666/FUJI16.pdf_2387490.pdf'),NaN,NaN,cc/000-02-004628/NIK241K.pdf_2375003.pdf,cc/000-01-858135/NIK241K.pdf_2173342.pdf,None,None,None,None,None,None,None,None
1,('cc/000-02-007159/TAM70180T.pdf_2378637.pdf','cc/000-02-015686/TAM70180T.pdf_2387496.pdf'),NaN,NaN,cc/000-02-363746/Leica23.pdf_2845584.pdf,cc/000-02-377077/Leica23.pdf_2865741.pdf,cc/000-02-007168/Leica23.pdf_2378642.pdf,cc/000-02-371748/Leica23.pdf_2857971.pdf,None,None,None,None,None,None
2,('cc/000-02-386228/OLY75.pdf_2878379.pdf','cc/000-02-377080/OLY75.pdf_2865742.pdf','cc/000-02-371751/OLY75.pdf_2857972.pdf'),NaN,cc/000-01-437480/OLY12100ECT.pdf_1664877.pdf,cc/000-01-636253/OLY12100ECT.pdf_1902666.pdf,None,None,None,None,None,None,None,None
3,('cc/000-02-178984/CAN35.pdf_2590871.pdf','cc/000-02-181193/CAN35.pdf_2595179.pdf','cc/000-02-168170/CAN35.pdf_2578176.pdf'),NaN,cc/000-01-997705/OLY75_A.pdf_2368110.pdf,cc/000-02-420650/OLY75.pdf_2933963.pdf,cc/000-02-363752/OLY75.pdf_2845586.pdf,cc/000-02-444386/OLY75.pdf_2961552.pdf,cc/000-02-386228/OLY75.pdf_2878379.pdf,cc/000-02-377080/OLY75.pdf_2865742.pdf,cc/000-02-448471/OLY75.pdf_2969283.pdf,cc/000-02-007173/OLY75.pdf_2378643.pdf,cc/000-02-371751/OLY75.pdf_2857972.pdf,cc/000-01-913424/OLY75_A.pdf_2245677.pdf
4,('cc/000-02-549316/SIG1835_N.pdf_3123590.pdf','cc/000-02-548066/SIG1835_N.pdf_3121264.pdf'),NaN,NaN,cc/000-01-837220/Pana818.pdf_2140403.pdf,cc/000-01-871387/Pana818.pdf_2193002.pdf,None,None,None,None,None,None,None,None


In [28]:
result.shape

(63, 14)